In [43]:
! pip install google-cloud-aiplatform langchain==0.0.335 guardrails-ai pydantic typing rich --upgrade --user

  Using cached pydantic-2.5.0-py3-none-any.whl (407 kB)


In [1]:
import os
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

In [2]:
from pydantic import BaseModel, Field
from typing import List, Optional
from guardrails.validators import ValidRange, ValidChoices

doctors_notes = """49 y/o Male with chronic macular rash to face &amp; hair, worse in beard, eyebrows &amp; nares.
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream"""

prompt = """
Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

${doctors_notes}

${gr.complete_json_suffix_v2}
"""

class Symptom(BaseModel):
    symptom: str = Field(description="Symptom that a patient is experiencing")
    affected_area: str = Field(description="What part of the body the symptom is affecting", validators=[ValidChoices(choices=['head', 'neck', 'chest'], on_fail="reask")])

class Medication(BaseModel):
    medication: str = Field(description="Name of the medication the patient is taking")
    response: str = Field(description="How the patient is responding to the medication")

class PatientInfo(BaseModel):
    gender: str = Field(description="Patient's gender")
    age: int = Field(validators=[ValidRange(min=0, max=100, on_fail="fix")])
    symptoms: List[Symptom] = Field(description="Symptoms that the patient is currently experiencing. Each symptom should be classified into a separate item in the list.")
    current_meds: List[Medication] = Field(description="Medications the patient is currently taking and their response")

In [3]:
from rich import print
import guardrails as gd

# From pydantic:
guard = gd.Guard.from_pydantic(output_class=PatientInfo, prompt=prompt)

# From XML RAIL spec
# guard = gd.Guard.from_rail('getting_started.rail')

print(guard.prompt)

Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's 
information.

${doctors_notes}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="gender" description="Patient's gender"/>
    <integer name="age" format="valid-range: min=0 max=100"/>
    <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each symptom should be 
classified into a separate item in the list.">
        <object>
            <string name="symptom" description="Symptom that a patient is experiencing"/>
            <string name="affected_area" description="What part of the body the symptom is affecting" 
format="valid-choices: choices=['head', 'neck', 'chest']"/>
        </object>
    </list>
    <list name="current_meds" description="Medications the patient is currently taking and their response">
        <object>
            <string name="medication" description="Name of the medication the patient is taking"/>
            <string name="response" description="How the patient is responding to the medication"/>
        </object>
    </list>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

In [4]:
import vertexai
from langchain.llms import VertexAI

vertexai.init(project="cloud-sandbox-danielw", location="us-central1")
llm = VertexAI(
    model_name="text-bison",
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
)

raw_llm_output, validated_output = guard(
    llm,
    prompt_params={"doctors_notes": doctors_notes}
)

# Print the validated output from the LLM
print(guard.state.most_recent_call.tree)

/root/.local/lib/python3.10/site-packages/guardrails/prompt/instructions.py:34: UserWarning: Instructions do not have any variables, if you are migrating follow the new variable convention documented here: https://docs.guardrailsai.com/0-2-migration/
  warn(
/root/.local/lib/python3.10/site-packages/guardrails/prompt/prompt.py:25: UserWarning: Prompt does not have any variables, if you are migrating follow the new variable convention documented here: https://docs.guardrailsai.com/0-2-migration/
  warnings.warn(


Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │                                                                                                         │ │
│   │ │ Given the following doctor's notes about a patient, please extract a dictionary that contains the       │ │
│   │ │ patient's information.                                                                                  │ │
│   │ │                                                                                                         │ │
│   │ │ 49 y/o Male with chronic macular rash to face &amp; hair, worse in beard, eyebrows &amp; nares.         │ │
│   │ │ Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream                                    │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
│   │ │ it into.                                                                                                │ │
│   │ │                                                                                                         │ │
│   │ │ <output>                                                                                                │ │
│   │ │     <string name="gender" description="Patient's gender"/>                                              │ │
│   │ │     <integer name="age" format="valid-range: min=0 max=100"/>                                           │ │
│   │ │     <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each        │ │
│   │ │ symptom should be classified into a separate item in the list.">                                        │ │
│   │ │         <object>                                                                                        │ │
│   │ │             <string name="symptom" description="Symptom that a patient is experiencing"/>               │ │
│   │ │             <string name="affected_area" description="What part of the body the symptom is affecting"   │ │
│   │ │ format="valid-choices: choices=['head', 'neck', 'chest']"/>                                             │ │
│   │ │         </object>                                                                                       │ │
│   │ │     </list>                                                                                             │ │
│   │ │     <list name="current_meds" description="Medications the patient is currently taking and their        │ │
│   │ │ response">                                                                                              │ │
│   │ │         <object>                                                                                        │ │
│   │ │             <string name="medication" description="Name of the medication the patient is taking"/>      │ │
│   │ │             <string name="response" description="How the patient is responding to the medication"/>     │ │
│   │ │         </object>                                                                                       │ │
│   │ │     </list>                                                                                             │ │
│   │ │ </output>                                                                                               │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ │ ONLY return a valid JSON object (no other t

In [5]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=guard.prompt.escape(),
    input_variables=guard.prompt.variable_names,
)

text_prompt = prompt.format_prompt(doctors_notes=doctors_notes).to_string()
print(text_prompt)

response = llm(text_prompt)
print(response)

print(guard.parse(response))

Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's 
information.

$49 y/o Male with chronic macular rash to face &amp; hair, worse in beard, eyebrows &amp; nares.
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="gender" description="Patient's gender"/>
    <integer name="age" format="valid-range: min=0 max=100"/>
    <list name="symptoms" description="Symptoms that the patient is currently experiencing. Each symptom should be 
classified into a separate item in the list.">
        <object>
            <string name="symptom" description="Symptom that a patient is experiencing"/>
            <string name="affected_area" description="What part of the body the symptom is affecting" 
format="valid-choices: choices=['head', 'neck', 'chest']"/>
        </object>
    </list>
    <list name="current_meds" description="Medications the patient is currently taking and their response">
        <object>
            <string name="medication" description="Name of the medication the patient is taking"/>
            <string name="response" description="How the patient is responding to the medication"/>
        </object>
    </list>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

```json
{
    "gender": "Male",
    "age": 49,
    "symptoms": [
        {
            "symptom": "Rash",
            "affected_area": "Face, hair, beard, eyebrows, nares"
        },
        {
            "symptom": "Itchy",
            "affected_area": "Face, hair, beard, eyebrows, nares"
        },
        {
            "symptom": "Flaky",
            "affected_area": "Face, hair, beard, eyebrows, nares"
        },
        {
            "symptom": "Scaly",
            "affected_area": "Face, hair, beard, eyebrows, nares"
        }
    ],
    "current_meds": [
        {
            "medication": "OTC steroid cream",
            "response": "Moderate"
        }
    ]
}
```

{
    'gender': 'Male',
    'age': 49,
    'symptoms': [
        {'symptom': 'Rash', 'affected_area': None},
        {'symptom': 'Itchy', 'affected_area': None},
        {'symptom': 'Flaky', 'affected_area': None},
        {'symptom': 'Scaly', 'affected_area': None}
    ],
    'current_meds': [{'medication': 'OTC steroid cream', 'response': 'Moderate'}]
}